In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline



Dense Subgraphs
===============

*Technical details are available in the API documentation:* :doc:`/code/api/strawberryfields.apps.subgraph`

Graphs can be used to model a wide variety of concepts: social networks, financial markets,
biological networks, and many others. A common problem of interest is to find subgraphs that
contain a large number of connections between their nodes. These subgraphs may correspond to
communities in social networks, correlated assets in a market, or mutually influential proteins
in a biological network.

Mathematically, this task is known as the `dense subgraph problem
<https://en.wikipedia.org/wiki/Dense_subgraph>`__. The density of a $k$-node subgraph is equal
to the number of its edges divided by the maximum possible number of edges.
Identifying the densest graph of a given size, known as the densest-$k$ subgraph problem,
is `NP-Hard <https://en.wikipedia.org/wiki/NP-hardness>`__.


As shown in :cite:`arrazola2018using`, a defining feature of GBS is that when we encode a graph
into a GBS device, it samples dense subgraphs with high probability. This property can be
used to find dense subgraphs by sampling from a GBS device and postprocessing the outputs.
Let's take a look!

Finding dense subgraphs
-----------------------
The first step is to import all required modules. We'll need the :mod:`~.apps.data`
module to load pre-generated samples, the :mod:`~.apps.sample` module to postselect samples, the
:mod:`~.apps.subgraph` module to search for dense subgraphs, and the :mod:`~.apps.plot` module to
visualize the graphs. We'll also use Plotly which is required for the :mod:`~.apps.plot` module and
NetworkX for graph operations.



In [ ]:
from strawberryfields.apps import data, sample, subgraph, plot
import plotly
import networkx as nx

Here we'll study a 30-node graph with a planted 10-node graph, as considered in
:cite:`arrazola2018using`. The graph is generated by joining two Erdős–Rényi random graphs. The
first graph of 20 nodes is created with edge probability of 0.5. The second planted
graph is generated with edge probability of 0.875. The planted nodes are the last ten nodes in the
graph. The two graphs are joined by selecting 8 nodes at random from both graphs and adding an
edge between them. This graph has the sneaky property that even though the planted subgraph is the
densest of its size, its nodes have a lower average degree than the nodes in the rest of the
graph.

The :mod:`~.apps.data` module has pre-generated GBS samples from this graph. Let's load them,
postselect on samples with a large number of clicks, and convert them to subgraphs:



In [ ]:
planted = data.Planted()
postselected = sample.postselect(planted, 16, 30)
pl_graph = nx.to_networkx_graph(planted.adj)
samples = sample.to_subgraphs(postselected, pl_graph)
print(len(samples))

Not bad! We have more than 2000 samples to play with 😎. The planted subgraph is actually easy to
identify; it even appears clearly from the force-directed Kamada-Kawai algorithm that is used to
plot graphs in Strawberry Fields:



In [ ]:
sub = list(range(20, 30))
plot_graph = plot.graph(pl_graph, sub)
plotly.offline.plot(plot_graph, filename="planted.html")

.. raw:: html
    :file: ../../examples_apps/planted.html

<div class="alert alert-info"><h4>Note</h4><p>The command ``plotly.offline.plot()`` is used to display plots in the documentation. In
    practice, you can simply use ``plot_graph.show()`` to view the graph.</p></div>



A more interesting challenge is to find dense subgraphs of different sizes; it is often
useful to identify many high-density subgraphs, not just the densest ones. This is the purpose of
the :func:`~.subgraph.search` function in the :mod:`~.apps.subgraph` module: to identify
collections of dense subgraphs for a range of sizes. The output of this function is a
dictionary whose keys correspond to subgraph sizes within the specified range. The values in
the dictionary are the top subgraphs of that size and their corresponding density.



In [ ]:
dense = subgraph.search(samples, pl_graph, 8, 16, max_count=3)  # we look at top 3 densest subgraphs
for k in range(8, 17):
    print(dense[k][0])  # print only the densest subgraph of each size

From the results of the search we learn that, depending on their size, the densest subgraphs
belong to different regions of the graph: dense subgraphs of less than ten nodes are contained
within the planted subgraph, whereas larger dense subgraphs appear outside of the planted
subgraph. Smaller dense subgraphs can be cliques, characterized by having
maximum density of 1, while larger subgraphs are less dense. Let's see what the smallest and
largest subgraphs look like:



In [ ]:
densest_8 = plot.graph(pl_graph, dense[8][0][1])
densest_16 = plot.graph(pl_graph, dense[12][0][1])

plotly.offline.plot(densest_8, filename="densest_8.html")

.. raw:: html
    :file: ../../examples_apps/densest_8.html



In [ ]:
plotly.offline.plot(densest_16, filename="densest_16.html")

.. raw:: html
    :file: ../../examples_apps/densest_16.html



In principle there are different methods to postprocess GBS outputs to identify dense
subgraphs. For example, techniques for finding maximum cliques, included in the
:mod:`~.apps.clique` module could help provide initial subgraphs that can be resized to find
larger dense subgraphs. Such methods are hybrid algorithms combining the ability of GBS to
sample dense subgraphs with clever classical techniques. Can you think of your own hybrid
algorithm? 🤔

